# Python Cheatsheets - EFFICIENT CODING

While using Python, it is essential to code efficiently by;
- Writing clean, readable, fast, and efficient Python code
- Profiling the code for bottlenecks
- Eliminating bottlenecks and bad design patterns
The code should run with minimum possible completion time and with minimum possible resource consumption. The Zen of Pyrthon by Tim Peters is as below:

In [1]:
import timeit
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---
### 1) Build with Built-ins

Python provides various built-ins which should be preferred while performing basic operations.
- Built-in types: ```list```, ```tuple```, ```set```, ```dict```, etc.
- Built-in functions: ```print()```, ```len()```, ```range()```, ```round()```, ```enumerate()```, ```map()```, ```zip()```, etc.
- Built-in modules: ```os```, ```sys```, ```itertools```, ```collections```, ```math```, etc.

__```range()``` function:__ For large sequence of numbers, using range is more practical.

In [8]:
# explicitly creating a list
nums = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# creating a list by unpacking a range object
nums_range = [*range(1,10)]

if(nums == nums_range):
    print("Lists are same:", nums_range)

Lists are same: [1, 2, 3, 4, 5, 6, 7, 8, 9]


__```enumerate()``` function:__ For obtaining an indexed list, enumerate is more efficient. Four different version of creating an indexed list is shown below for a given list.

In [19]:
# generate a list to be indexed
chapters = ['Introduction', 'Related Work', 'Methodology', 'Results', 'Discussion', 'Conclusion']

In [21]:
%%timeit
# write with a basic for loop
indexed_chapters = []
for i in range(len(chapters)):
    indexed_chapters.append((i, chapters[i]))

1.41 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
%%timeit
# loop with enumerate
indexed_chapters_enum = []
for i, chapter in enumerate(chapters):
    indexed_chapters_enum.append((i,chapter))

1.33 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [23]:
%%timeit
# loop with enumerate using list comprehension
indexed_chapters_comp = [(i, chapter) for i, chapter in enumerate(chapters)]

1.12 µs ± 36.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:
%%timeit
# unpacking enumerate object with a starting index of one (FASTEST)
indexed_chapters_unpack = [*enumerate(chapters, 1)]

514 ns ± 32 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
